In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
%matplotlib inline

In [2]:
training = pd.read_csv('processedTrain.csv')

In [3]:
from imblearn.over_sampling import SMOTE
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import BorderlineSMOTE
from imblearn.over_sampling import SVMSMOTE 
from imblearn.combine import SMOTEENN 
from imblearn.combine import SMOTETomek 
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler 

In [4]:
sm = BorderlineSMOTE(random_state=2021,kind='borderline-2')
ros = RandomOverSampler(sampling_strategy="not majority",random_state=2021)

In [5]:
from pycaret.classification import *

In [6]:
classify = setup(data = training , target = 'AVERAGE_SPEED_DIFF',train_size=0.8,session_id=2021,normalize = True,fix_imbalance=True,fix_imbalance_method=sm)

,Description,Value
0,session_id,2021
1,Target,AVERAGE_SPEED_DIFF
2,Target Type,Multiclass
3,Label Encoded,"High: 0, Low: 1, Medium: 2, None: 3, Very_High: 4"
4,Original Data,"(6812, 30)"
5,Missing Values,False
6,Numeric Features,20
7,Categorical Features,9
8,Ordinal Features,False
9,High Cardinality Features,False


In [7]:
best_model = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
lightgbm,Light Gradient Boosting Machine,0.8121,0.9649,0.8114,0.8150,0.8127,0.7549,0.7553,0.5260
xgboost,Extreme Gradient Boosting,0.8106,0.9638,0.8076,0.8136,0.8112,0.7529,0.7533,1.3560
rf,Random Forest Classifier,0.8053,0.9647,0.8090,0.8099,0.8058,0.7464,0.7473,0.2700
gbc,Gradient Boosting Classifier,0.7985,0.9632,0.8026,0.8023,0.7989,0.7376,0.7383,1.6170
et,Extra Trees Classifier,0.7976,0.9628,0.7985,0.7995,0.7971,0.7356,0.7363,0.2130
lr,Logistic Regression,0.7651,0.9519,0.7729,0.7725,0.7644,0.6948,0.6964,0.4460
lda,Linear Discriminant Analysis,0.7221,0.9263,0.7349,0.7206,0.7170,0.6376,0.6392,0.0970
dt,Decision Tree Classifier,0.7201,0.8210,0.7175,0.7245,0.7210,0.6356,0.6362,0.1020
nb,Naive Bayes,0.7130,0.9200,0.7211,0.7148,0.7072,0.6256,0.6280,0.0930
svm,SVM - Linear Kernel,0.6711,0.0000,0.6747,0.6722,0.6594,0.5720,0.5767,0.1170


In [8]:
lightgbm = create_model('lightgbm')
rf = create_model('rf')
xgboost = create_model('xgboost')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8092,0.9629,0.8156,0.8114,0.8093,0.7514,0.7518
1,0.8000,0.9583,0.7936,0.8055,0.8015,0.7388,0.7394
2,0.7963,0.9607,0.7912,0.7953,0.7957,0.7332,0.7333
3,0.8165,0.9681,0.8199,0.8225,0.8175,0.7610,0.7618
4,0.7890,0.9577,0.7764,0.7949,0.7901,0.7251,0.7260
5,0.8239,0.9658,0.8180,0.8244,0.8240,0.7697,0.7698
6,0.8147,0.9657,0.8040,0.8203,0.8163,0.7582,0.7589
7,0.8128,0.9660,0.8112,0.8140,0.8131,0.7558,0.7559
8,0.8147,0.9638,0.8072,0.8186,0.8160,0.7585,0.7589
9,0.8290,0.9684,0.8389,0.8297,0.8287,0.7768,0.7772


In [9]:
bagged_lightgbm = ensemble_model(lightgbm, method = 'Bagging')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8147,0.9631,0.8233,0.8168,0.8148,0.7585,0.7590
1,0.7945,0.9607,0.7896,0.8020,0.7964,0.7321,0.7330
2,0.7927,0.9628,0.7850,0.7914,0.7914,0.7284,0.7288
3,0.8202,0.9690,0.8291,0.8237,0.8209,0.7660,0.7664
4,0.7927,0.9621,0.7879,0.7988,0.7943,0.7302,0.7309
5,0.8055,0.9672,0.8072,0.8060,0.8055,0.7460,0.7461
6,0.8073,0.9672,0.7988,0.8120,0.8090,0.7486,0.7490
7,0.8239,0.9680,0.8252,0.8267,0.8245,0.7705,0.7709
8,0.8275,0.9691,0.8220,0.8299,0.8282,0.7752,0.7754
9,0.8180,0.9700,0.8249,0.8204,0.8179,0.7626,0.7633


In [10]:
bagged_rf = ensemble_model(rf, method = 'Bagging')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7945,0.9628,0.8110,0.8050,0.7956,0.7334,0.7356
1,0.7908,0.9601,0.7883,0.8054,0.7935,0.7281,0.7308
2,0.7945,0.9633,0.8008,0.7954,0.7938,0.7317,0.7323
3,0.8128,0.9695,0.8233,0.8199,0.8139,0.7570,0.7580
4,0.7853,0.9625,0.7876,0.7941,0.7863,0.7210,0.7228
5,0.8073,0.9647,0.8171,0.8076,0.8067,0.7488,0.7492
6,0.7927,0.9658,0.7803,0.7993,0.7947,0.7297,0.7304
7,0.8018,0.9650,0.8114,0.8072,0.8019,0.7423,0.7435
8,0.8018,0.9682,0.8058,0.8136,0.8032,0.7433,0.7454
9,0.8162,0.9682,0.8301,0.8215,0.8165,0.7607,0.7619


In [11]:
bagged_xgb = ensemble_model(xgboost, method = 'Bagging')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7982,0.9626,0.8078,0.8000,0.7983,0.7372,0.7375
1,0.8110,0.9621,0.8073,0.8176,0.8126,0.7535,0.7543
2,0.8000,0.9635,0.7928,0.8003,0.7992,0.7382,0.7387
3,0.8165,0.9704,0.8224,0.8198,0.8168,0.7610,0.7615
4,0.7908,0.9609,0.7848,0.7968,0.7919,0.7276,0.7286
5,0.7982,0.9667,0.7992,0.7976,0.7974,0.7362,0.7364
6,0.8037,0.9666,0.7945,0.8085,0.8054,0.7438,0.7442
7,0.8165,0.9660,0.8222,0.8192,0.8169,0.7610,0.7615
8,0.8202,0.9673,0.8146,0.8225,0.8209,0.7656,0.7658
9,0.8217,0.9689,0.8325,0.8251,0.8216,0.7676,0.7685


In [12]:
blender_specific = blend_models(estimator_list = [lightgbm,rf,xgboost], method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8110,0.9642,0.8186,0.8133,0.8113,0.7537,0.7542
1,0.7890,0.9615,0.7852,0.7959,0.7905,0.7249,0.7257
2,0.7982,0.9641,0.7900,0.7975,0.7974,0.7357,0.7359
3,0.8275,0.9707,0.8309,0.8318,0.8284,0.7755,0.7760
4,0.8000,0.9615,0.7902,0.8061,0.8016,0.7395,0.7403
5,0.8092,0.9675,0.8113,0.8100,0.8091,0.7509,0.7511
6,0.8165,0.9670,0.8042,0.8196,0.8175,0.7603,0.7606
7,0.8128,0.9677,0.8077,0.8154,0.8136,0.7559,0.7562
8,0.8312,0.9682,0.8248,0.8342,0.8321,0.7801,0.7803
9,0.8309,0.9716,0.8405,0.8312,0.8301,0.7792,0.7797


In [26]:
evaluate_model(lightgbm)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…

In [14]:
predict_model(bagged_lightgbm);

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Light Gradient Boosting Machine,0.8085,0.9664,0.8082,0.8123,0.8099,0.7491,0.7494


In [15]:
final_rf = finalize_model(bagged_lightgbm)

In [16]:
testing = pd.read_csv('processedTest.csv')
unseen_predictions = predict_model(bagged_lightgbm, data=testing)

In [17]:
unseen_predictions['AVERAGE_SPEED_DIFF'] = unseen_predictions['Label']

In [18]:
sub_df = pd.read_csv('example_submission.csv')
final_data = {'RowId': sub_df.RowId, 'Speed_Diff': unseen_predictions['AVERAGE_SPEED_DIFF']}
submission = pd.DataFrame(data=final_data)

In [19]:
submission['Speed_Diff'].value_counts()

None         425
Low          359
Medium       340
High         262
Very_High    114
Name: Speed_Diff, dtype: int64

In [20]:
compare = pd.read_csv('sample_submission.csv')

In [21]:
compare['Speed_Diff'].value_counts()

None         482
Medium       340
Low          313
High         268
Very_High     97
Name: Speed_Diff, dtype: int64

In [22]:
df1 = np.where(submission['Speed_Diff'] == compare['Speed_Diff'], 'True', 'False')
df1 = pd.DataFrame(df1)
df1.columns=['equals']
df1.value_counts()

equals
True      1193
False      307
dtype: int64

In [23]:
submission.head()

,RowId,Speed_Diff
0,1,None
1,2,Medium
2,3,None
3,4,Medium
4,5,Low


In [24]:
submission.to_csv('submission.csv',index=False)